In [ ]:
# ============================================================
# GENERAL-PURPOSE NAS FOR TABULAR REGRESSION
# Works for: AgroPlanner, INS Calibration, Sensor Modeling
# ============================================================

# -------------------- IMPORTS --------------------
import torch
import torch.nn as nn
import torch.optim as optim
import optuna
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from torch.utils.data import DataLoader, TensorDataset

# -------------------- DEVICE --------------------
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
torch.backends.cudnn.benchmark = True

# ============================================================
# 1. USER CONFIG (ONLY CHANGE THIS FOR NEW PROJECTS)
# ============================================================

CSV_PATH = "dataset.csv"          # Your dataset
TARGET_COLUMN = "yield"           # e.g. "yield", "bias_x", "scale_y"
CATEGORICAL_COLUMNS = ["dist_name", "crop_type"]  # [] for INS
TEST_SIZE = 0.15
VAL_SIZE = 0.15

NAS_CONFIG = {
    "epochs": 80,
    "batch_size": 128,
    "trials": 30,
    "patience": 8
}

# ============================================================
# 2. DATA LOADING & PREPROCESSING
# ============================================================

df = pd.read_csv(CSV_PATH)

# Encode categorical features
for col in CATEGORICAL_COLUMNS:
    le = LabelEncoder()
    df[col] = le.fit_transform(df[col])

X = df.drop(columns=[TARGET_COLUMN]).values
y = df[TARGET_COLUMN].values.reshape(-1, 1)

# Standardization (CRITICAL for INS)
scaler = StandardScaler()
X = scaler.fit_transform(X)

# Split: Train / Val / Test
X_train, X_temp, y_train, y_temp = train_test_split(
    X, y, test_size=(TEST_SIZE + VAL_SIZE), random_state=42
)

relative_val_size = VAL_SIZE / (TEST_SIZE + VAL_SIZE)

X_val, X_test, y_val, y_test = train_test_split(
    X_temp, y_temp, test_size=relative_val_size, random_state=42
)

def to_tensor(x, y):
    return torch.tensor(x, dtype=torch.float32), torch.tensor(y, dtype=torch.float32)

X_train, y_train = to_tensor(X_train, y_train)
X_val, y_val = to_tensor(X_val, y_val)
X_test, y_test = to_tensor(X_test, y_test)

train_loader = DataLoader(
    TensorDataset(X_train, y_train),
    batch_size=NAS_CONFIG["batch_size"],
    shuffle=True,
    pin_memory=True
)

val_loader = DataLoader(
    TensorDataset(X_val, y_val),
    batch_size=NAS_CONFIG["batch_size"],
    pin_memory=True
)

test_loader = DataLoader(
    TensorDataset(X_test, y_test),
    batch_size=NAS_CONFIG["batch_size"],
    pin_memory=True
)

INPUT_DIM = X_train.shape[1]

# ============================================================
# 3. NAS MODEL
# ============================================================

class NASNet(nn.Module):
    def __init__(self, input_dim, layers, activation, dropout):
        super().__init__()
        net = []
        prev = input_dim

        for h in layers:
            net += [
                nn.Linear(prev, h),
                nn.BatchNorm1d(h),
                activation(),
                nn.Dropout(dropout)
            ]
            prev = h

        net.append(nn.Linear(prev, 1))
        self.model = nn.Sequential(*net)

    def forward(self, x):
        return self.model(x)

# ============================================================
# 4. METRICS
# ============================================================

def compute_metrics(y_true, y_pred):
    mse = mean_squared_error(y_true, y_pred)
    rmse = np.sqrt(mse)
    mae = mean_absolute_error(y_true, y_pred)
    r2 = r2_score(y_true, y_pred)
    return mse, rmse, mae, r2

def evaluate(model, loader):
    model.eval()
    y_true, y_pred = [], []

    with torch.no_grad():
        for x, y in loader:
            preds = model(x.to(DEVICE)).cpu().numpy()
            y_true.extend(y.numpy())
            y_pred.extend(preds)

    return compute_metrics(np.array(y_true), np.array(y_pred))

# ============================================================
# 5. TRAIN LOOP
# ============================================================

def train_epoch(model, optimizer, criterion):
    model.train()
    for x, y in train_loader:
        optimizer.zero_grad()
        loss = criterion(model(x.to(DEVICE)), y.to(DEVICE))
        loss.backward()
        optimizer.step()

# ============================================================
# 6. NAS OBJECTIVE
# ============================================================

def objective(trial):

    n_layers = trial.suggest_int("n_layers", 1, 4)
    layers = [
        trial.suggest_int(f"units_l{i}", 32, 192, step=32)
        for i in range(n_layers)
    ]

    activation_name = trial.suggest_categorical(
        "activation", ["ReLU", "LeakyReLU", "ELU"]
    )
    activation = {
        "ReLU": nn.ReLU,
        "LeakyReLU": nn.LeakyReLU,
        "ELU": nn.ELU
    }[activation_name]

    dropout = trial.suggest_float("dropout", 0.1, 0.5)
    lr = trial.suggest_float("lr", 1e-4, 1e-2, log=True)
    wd = trial.suggest_float("weight_decay", 1e-6, 1e-3, log=True)

    model = NASNet(INPUT_DIM, layers, activation, dropout).to(DEVICE)
    optimizer = optim.Adam(model.parameters(), lr=lr, weight_decay=wd)
    criterion = nn.MSELoss()

    best_val = float("inf")
    patience = 0

    for epoch in range(NAS_CONFIG["epochs"]):
        train_epoch(model, optimizer, criterion)
        val_mse, _, _, _ = evaluate(model, val_loader)

        trial.report(val_mse, epoch)
        if trial.should_prune():
            raise optuna.exceptions.TrialPruned()

        if val_mse < best_val:
            best_val = val_mse
            patience = 0
        else:
            patience += 1
        if patience >= NAS_CONFIG["patience"]:
            break

    val_mse, val_rmse, val_mae, val_r2 = evaluate(model, val_loader)

    print(
        f"Trial {trial.number} | "
        f"MSE: {val_mse:.4f}, RMSE: {val_rmse:.4f}, "
        f"MAE: {val_mae:.4f}, R2: {val_r2:.4f}"
    )

    trial.set_user_attr("MSE", val_mse)
    trial.set_user_attr("RMSE", val_rmse)
    trial.set_user_attr("MAE", val_mae)
    trial.set_user_attr("R2", val_r2)

    return val_mse

# ============================================================
# 7. RUN NAS
# ============================================================

study = optuna.create_study(
    direction="minimize",
    pruner=optuna.pruners.MedianPruner(n_warmup_steps=5)
)
study.optimize(objective, n_trials=NAS_CONFIG["trials"])

# ============================================================
# 8. BEST MODEL EVALUATION
# ============================================================

params = study.best_params

best_layers = [params[f"units_l{i}"] for i in range(params["n_layers"])]
best_activation = {
    "ReLU": nn.ReLU,
    "LeakyReLU": nn.LeakyReLU,
    "ELU": nn.ELU
}[params["activation"]]

final_model = NASNet(
    INPUT_DIM, best_layers, best_activation, params["dropout"]
).to(DEVICE)

optimizer = optim.Adam(
    final_model.parameters(),
    lr=params["lr"],
    weight_decay=params["weight_decay"]
)

criterion = nn.MSELoss()

for _ in range(NAS_CONFIG["epochs"]):
    train_epoch(final_model, optimizer, criterion)

print("\n====== FINAL MODEL METRICS ======")
print("TRAIN:", evaluate(final_model, train_loader))
print("VAL  :", evaluate(final_model, val_loader))
print("TEST :", evaluate(final_model, test_loader))

torch.save(final_model.state_dict(), "best_nas_model.pth")
print("\n✔ Saved best_nas_model.pth")


In [ ]:
# ============================================================
# AgroPlanner – Neural Architecture Search with Full Metrics
# ============================================================

# -------------------- IMPORTS --------------------
import torch
import torch.nn as nn
import torch.optim as optim
import optuna
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from torch.utils.data import DataLoader, TensorDataset

# -------------------- GPU CONFIG --------------------
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
torch.backends.cudnn.benchmark = True

# -------------------- NAS CONFIG --------------------
EPOCHS = 80
BATCH_SIZE = 128
N_TRIALS = 30
PATIENCE = 8

# -------------------- LOAD DATA --------------------
df = pd.read_csv("dataset.csv")

TARGET = "yield"
CATEGORICAL_COLS = ["dist_name", "crop_type"]
NUMERIC_COLS = [c for c in df.columns if c not in CATEGORICAL_COLS + [TARGET]]

# Encode categorical columns
for col in CATEGORICAL_COLS:
    le = LabelEncoder()
    df[col] = le.fit_transform(df[col])

X = df[CATEGORICAL_COLS + NUMERIC_COLS].values
y = df[TARGET].values.reshape(-1, 1)

# Standardize
scaler = StandardScaler()
X = scaler.fit_transform(X)

# -------------------- SPLITS --------------------
# 70% Train, 15% Val, 15% Test
X_train, X_temp, y_train, y_temp = train_test_split(
    X, y, test_size=0.3, random_state=42
)
X_val, X_test, y_val, y_test = train_test_split(
    X_temp, y_temp, test_size=0.5, random_state=42
)

# -------------------- TENSORS --------------------
def to_tensor(x, y):
    return (
        torch.tensor(x, dtype=torch.float32),
        torch.tensor(y, dtype=torch.float32)
    )

X_train, y_train = to_tensor(X_train, y_train)
X_val, y_val     = to_tensor(X_val, y_val)
X_test, y_test   = to_tensor(X_test, y_test)

train_loader = DataLoader(
    TensorDataset(X_train, y_train),
    batch_size=BATCH_SIZE,
    shuffle=True,
    pin_memory=True
)

val_loader = DataLoader(
    TensorDataset(X_val, y_val),
    batch_size=BATCH_SIZE,
    pin_memory=True
)

test_loader = DataLoader(
    TensorDataset(X_test, y_test),
    batch_size=BATCH_SIZE,
    pin_memory=True
)

INPUT_DIM = X_train.shape[1]
OUTPUT_DIM = 1

# -------------------- MODEL --------------------
class NASNet(nn.Module):
    def __init__(self, input_dim, layers, activation, dropout):
        super().__init__()
        modules = []
        prev_dim = input_dim

        for h in layers:
            modules.append(nn.Linear(prev_dim, h))
            modules.append(nn.BatchNorm1d(h))
            modules.append(activation())
            modules.append(nn.Dropout(dropout))
            prev_dim = h

        modules.append(nn.Linear(prev_dim, OUTPUT_DIM))
        self.net = nn.Sequential(*modules)

    def forward(self, x):
        return self.net(x)

# -------------------- METRICS --------------------
def compute_metrics(y_true, y_pred):
    mse = mean_squared_error(y_true, y_pred)
    rmse = np.sqrt(mse)
    mae = mean_absolute_error(y_true, y_pred)
    r2 = r2_score(y_true, y_pred)
    return mse, rmse, mae, r2

def evaluate_model(model, loader):
    model.eval()
    y_true, y_pred = [], []

    with torch.no_grad():
        for x, y in loader:
            x = x.to(DEVICE)
            preds = model(x).cpu().numpy()
            y_true.extend(y.numpy())
            y_pred.extend(preds)

    return compute_metrics(np.array(y_true), np.array(y_pred))

# -------------------- TRAIN LOOP --------------------
def train_epoch(model, optimizer, criterion):
    model.train()
    for x, y in train_loader:
        x = x.to(DEVICE, non_blocking=True)
        y = y.to(DEVICE, non_blocking=True)

        optimizer.zero_grad()
        loss = criterion(model(x), y)
        loss.backward()
        optimizer.step()

# -------------------- OPTUNA OBJECTIVE --------------------
def objective(trial):

    n_layers = trial.suggest_int("n_layers", 1, 4)
    layers = [
        trial.suggest_int(f"units_l{i}", 32, 192, step=32)
        for i in range(n_layers)
    ]

    activation_name = trial.suggest_categorical(
        "activation", ["ReLU", "LeakyReLU", "ELU"]
    )
    activation = {
        "ReLU": nn.ReLU,
        "LeakyReLU": nn.LeakyReLU,
        "ELU": nn.ELU
    }[activation_name]

    dropout = trial.suggest_float("dropout", 0.1, 0.5)
    lr = trial.suggest_float("lr", 1e-4, 1e-2, log=True)
    weight_decay = trial.suggest_float("weight_decay", 1e-6, 1e-3, log=True)

    model = NASNet(INPUT_DIM, layers, activation, dropout).to(DEVICE)
    optimizer = optim.Adam(model.parameters(), lr=lr, weight_decay=weight_decay)
    criterion = nn.MSELoss()

    best_val = float("inf")
    patience = 0

    for epoch in range(EPOCHS):
        train_epoch(model, optimizer, criterion)
        val_mse, _, _, _ = evaluate_model(model, val_loader)

        trial.report(val_mse, epoch)
        if trial.should_prune():
            raise optuna.exceptions.TrialPruned()

        if val_mse < best_val:
            best_val = val_mse
            patience = 0
        else:
            patience += 1
        if patience >= PATIENCE:
            break

    val_mse, val_rmse, val_mae, val_r2 = evaluate_model(model, val_loader)

    print(
        f"Trial {trial.number} | "
        f"MSE: {val_mse:.4f} | RMSE: {val_rmse:.4f} | "
        f"MAE: {val_mae:.4f} | R2: {val_r2:.4f}"
    )

    trial.set_user_attr("MSE", val_mse)
    trial.set_user_attr("RMSE", val_rmse)
    trial.set_user_attr("MAE", val_mae)
    trial.set_user_attr("R2", val_r2)

    return val_mse

# -------------------- RUN NAS --------------------
study = optuna.create_study(
    direction="minimize",
    pruner=optuna.pruners.MedianPruner(n_warmup_steps=5)
)
study.optimize(objective, n_trials=N_TRIALS)

# -------------------- PRINT ALL RESULTS --------------------
print("\n========= ALL NAS TRIAL RESULTS =========")
for t in study.trials:
    if t.state.name == "COMPLETE":
        print(
            f"Trial {t.number} | "
            f"MSE: {t.user_attrs['MSE']:.4f}, "
            f"RMSE: {t.user_attrs['RMSE']:.4f}, "
            f"MAE: {t.user_attrs['MAE']:.4f}, "
            f"R2: {t.user_attrs['R2']:.4f}"
        )

# -------------------- BEST MODEL --------------------
params = study.best_params

best_layers = [params[f"units_l{i}"] for i in range(params["n_layers"])]
best_activation = {
    "ReLU": nn.ReLU,
    "LeakyReLU": nn.LeakyReLU,
    "ELU": nn.ELU
}[params["activation"]]

final_model = NASNet(
    INPUT_DIM, best_layers, best_activation, params["dropout"]
).to(DEVICE)

optimizer = optim.Adam(
    final_model.parameters(),
    lr=params["lr"],
    weight_decay=params["weight_decay"]
)
criterion = nn.MSELoss()

for _ in range(EPOCHS):
    train_epoch(final_model, optimizer, criterion)

# -------------------- FINAL METRICS --------------------
train_metrics = evaluate_model(final_model, train_loader)
val_metrics   = evaluate_model(final_model, val_loader)
test_metrics  = evaluate_model(final_model, test_loader)

print("\n========= BEST MODEL METRICS =========")

print("TRAIN:")
print(f"MSE: {train_metrics[0]:.4f}, RMSE: {train_metrics[1]:.4f}, "
      f"MAE: {train_metrics[2]:.4f}, R2: {train_metrics[3]:.4f}")

print("\nVALIDATION:")
print(f"MSE: {val_metrics[0]:.4f}, RMSE: {val_metrics[1]:.4f}, "
      f"MAE: {val_metrics[2]:.4f}, R2: {val_metrics[3]:.4f}")

print("\nTEST:")
print(f"MSE: {test_metrics[0]:.4f}, RMSE: {test_metrics[1]:.4f}, "
      f"MAE: {test_metrics[2]:.4f}, R2: {test_metrics[3]:.4f}")

# -------------------- SAVE MODEL --------------------
torch.save(final_model.state_dict(), "best_nas_model.pth")
print("\n✔ Best NAS model saved as best_nas_model.pth")


In [ ]:
import torch
from torch.profiler import profile, record_function, ProfilerActivity

model.train()

with profile(
    activities=[ProfilerActivity.CPU, ProfilerActivity.CUDA],
    record_shapes=True,
    profile_memory=True,
    with_stack=True
) as prof:

    for batch_idx, (x, y) in enumerate(train_loader):
        if batch_idx >= 5:   # profile first 5 batches only
            break

        x, y = x.to(device), y.to(device)

        with record_function("forward"):
            outputs = model(x)
            loss = criterion(outputs, y)

        with record_function("backward"):
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

print(
    prof.key_averages()
    .table(sort_by="cuda_time_total", row_limit=15)
)
